# IMPORTS

In [3]:
import numpy as np
from astropy.io import fits
import astropy.io.fits as pyfits
from astropy.wcs import WCS
import matplotlib.pyplot as plt
from astropy.table import Table
import pandas as pd
import math
import random as r
import scipy.stats
import os.path
import sys
sys.path.append('/home/machado.35/projects/modules')
from deprojectGalaxy import deproject
import time
import spacing_calc

# SETUP

In [4]:
sources = pd.read_csv('/home/machado.35/projects/intro/scripts/sources.csv')
sources = list(sources['sources'])
res = [60,90,120,150]

tab_loc = '/home/machado.35/projects/intro/phangs_sample_table_v1p6.fits'
t = Table.read(tab_loc)

output_path = '/data/rubin/machado.35/phangs/rand_results/final_run/'

# RUN RANDOM PLACEMENTS

In [10]:
#2D COVERAGE
n_clouds = 1e6
for k in range(len(res)):
    for i in range(len(sources)):
        image = '/home/machado.35/projects/masks/coverage/'+sources[i]+'_12m+7m+tp_co21_'+str(res[k])+'pc_coverage2d.fits'
        if os.path.isfile(image):
            row = int(np.where(t['name'] == sources[i])[0])
            galRA=t[row]['orient_ra']
            galDEC=t[row]['orient_dec']
            pa=t[row]['orient_posang']
            incl=t[row]['orient_incl']
            galDist=t[row]['dist']
            outfile_name = output_path+sources[i]+'_2dcov'+'_'+str(res[k])+'pc.csv'
            spacing_calc.place_nclouds_in_map(image, n_clouds, outfile_name, galRA, galDEC, pa, incl, galDist)

In [11]:
#EXP MASK
n_clouds = 1e6
for k in range(len(res)):
    for i in range(len(sources)):
        image = '/home/machado.35/projects/masks/exp/'+sources[i]+'_'+str(res[k])+'pc_exp_mask.fits'
        if os.path.isfile(image):
            row = int(np.where(t['name'] == sources[i])[0])
            galRA=t[row]['orient_ra']
            galDEC=t[row]['orient_dec']
            pa=t[row]['orient_posang']
            incl=t[row]['orient_incl']
            galDist=t[row]['dist']
            outfile_name = output_path+sources[i]+'_exp'+'_'+str(res[k])+'pc.csv'
            spacing_calc.place_nclouds_in_map(image, n_clouds, outfile_name, galRA, galDEC, pa, incl, galDist)

# CALCULATE 1ST, 3RD, 5TH NEIGHBORS & 400, 600, 800, 1000 PC

In [5]:
#2D COVERAGE
radii = [100,200,400,600,800,1000,1500]
for k in range(len(res)):
    for i in range(len(sources)):
        image = '/home/machado.35/projects/masks/coverage/'+sources[i]+'_12m+7m+tp_co21_'+str(res[k])+'pc_coverage2d.fits'
        cat_fp = '/home/machado.35/projects/intro/sources/'+sources[i]+'/'+sources[i]+'_'+str(res[k])+'pc_cloud_stats.csv'
        fn = output_path+sources[i]+'_2dcov'+'_'+str(res[k])+'pc.csv'
        if os.path.isfile(image):
            if os.path.isfile(cat_fp):
                if os.path.isfile(fn):
                    row = int(np.where(t['name'] == sources[i])[0])
                    dist = t['dist'][row]*1e6 #from Mpc to pc
                    df = pd.read_csv(fn)
                    cat = pd.read_csv(cat_fp)
                    x = df['rRA']
                    y = df['rDEC']
                    N_tot = len(x)
                    n_it = len(cat)
                    n_loops = int(np.floor(N_tot / n_it))
                    mindist = np.zeros_like(x)*np.nan
                    mindist3 = np.zeros_like(x)*np.nan
                    mindist5 = np.zeros_like(x)*np.nan
                    radial_density_100 = np.zeros_like(x)*np.nan
                    radial_density_200 = np.zeros_like(x)*np.nan
                    radial_density_400 = np.zeros_like(x)*np.nan
                    radial_density_600 = np.zeros_like(x)*np.nan
                    radial_density_800 = np.zeros_like(x)*np.nan
                    radial_density_1000 = np.zeros_like(x)*np.nan
                    radial_density_1500 = np.zeros_like(x)*np.nan
                    for ii in np.arange(n_loops):
                        tx = x[ii*n_it:(ii+1)*n_it]
                        ty = y[ii*n_it:(ii+1)*n_it]
                        mindist[ii*n_it:(ii+1)*n_it] = spacing_calc.pair_dist_calc(x=tx,y=ty, neighbor_num=1) #1st nearest
                        mindist3[ii*n_it:(ii+1)*n_it] = spacing_calc.pair_dist_calc(x=tx,y=ty, neighbor_num=3) #3rd nearest
                        mindist5[ii*n_it:(ii+1)*n_it] = spacing_calc.pair_dist_calc(x=tx,y=ty, neighbor_num=5) #5th nearest
                        radial_density_100[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[0], galdist=dist)
                        radial_density_200[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[1], galdist=dist)
                        radial_density_400[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[2], galdist=dist)
                        radial_density_600[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[3], galdist=dist)
                        radial_density_800[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[4], galdist=dist)
                        radial_density_1000[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[5], galdist=dist)
                        radial_density_1500[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[6], galdist=dist)
                    df['nn_dist'] = np.deg2rad(mindist) * dist
                    df['nn3_dist'] = np.deg2rad(mindist3) * dist
                    df['nn5_dist'] = np.deg2rad(mindist5) * dist
                    df['clouds_r'+str(radii[0])+'pc'] = radial_density_100
                    df['clouds_r'+str(radii[1])+'pc'] = radial_density_200
                    df['clouds_r'+str(radii[2])+'pc'] = radial_density_400
                    df['clouds_r'+str(radii[3])+'pc'] = radial_density_600
                    df['clouds_r'+str(radii[4])+'pc'] = radial_density_800
                    df['clouds_r'+str(radii[5])+'pc'] = radial_density_1000
                    df['clouds_r'+str(radii[6])+'pc'] = radial_density_1500
                    df.to_csv(fn)

In [6]:
#EXP DISK
radii = [100,200,400,600,800,1000,1500]
for k in range(len(res)):
    for i in range(len(sources)):
        image = '/home/machado.35/projects/masks/exp/'+sources[i]+'_'+str(res[k])+'pc_exp_mask.fits'
        cat_fp = '/home/machado.35/projects/intro/sources/'+sources[i]+'/'+sources[i]+'_'+str(res[k])+'pc_cloud_stats.csv'
        fn = output_path+sources[i]+'_exp'+'_'+str(res[k])+'pc.csv'
        if os.path.isfile(image):
            if os.path.isfile(cat_fp):
                if os.path.isfile(fn):
                    row = int(np.where(t['name'] == sources[i])[0])
                    dist = t['dist'][row]*1e6 #from Mpc to pc
                    df = pd.read_csv(fn)
                    cat = pd.read_csv(cat_fp)
                    x = df['rRA']
                    y = df['rDEC']
                    N_tot = len(x)
                    n_it = len(cat)
                    n_loops = int(np.floor(N_tot / n_it))
                    mindist = np.zeros_like(x)*np.nan
                    mindist3 = np.zeros_like(x)*np.nan
                    mindist5 = np.zeros_like(x)*np.nan
                    radial_density_100 = np.zeros_like(x)*np.nan
                    radial_density_200 = np.zeros_like(x)*np.nan
                    radial_density_400 = np.zeros_like(x)*np.nan
                    radial_density_600 = np.zeros_like(x)*np.nan
                    radial_density_800 = np.zeros_like(x)*np.nan
                    radial_density_1000 = np.zeros_like(x)*np.nan
                    radial_density_1500 = np.zeros_like(x)*np.nan
                    for ii in np.arange(n_loops):
                        tx = x[ii*n_it:(ii+1)*n_it]
                        ty = y[ii*n_it:(ii+1)*n_it]
                        mindist[ii*n_it:(ii+1)*n_it] = spacing_calc.pair_dist_calc(x=tx,y=ty, neighbor_num=1) #1st nearest
                        mindist3[ii*n_it:(ii+1)*n_it] = spacing_calc.pair_dist_calc(x=tx,y=ty, neighbor_num=3) #3rd nearest
                        mindist5[ii*n_it:(ii+1)*n_it] = spacing_calc.pair_dist_calc(x=tx,y=ty, neighbor_num=5) #5th nearest
                        radial_density_100[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[0], galdist=dist)
                        radial_density_200[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[1], galdist=dist)
                        radial_density_400[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[2], galdist=dist)
                        radial_density_600[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[3], galdist=dist)
                        radial_density_800[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[4], galdist=dist)
                        radial_density_1000[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[5], galdist=dist)
                        radial_density_1500[ii*n_it:(ii+1)*n_it] = spacing_calc.rad_density(x=tx,y=ty, rad=radii[6], galdist=dist)
                    df['nn_dist'] = np.deg2rad(mindist) * dist
                    df['nn3_dist'] = np.deg2rad(mindist3) * dist
                    df['nn5_dist'] = np.deg2rad(mindist5) * dist
                    df['clouds_r'+str(radii[0])+'pc'] = radial_density_100
                    df['clouds_r'+str(radii[1])+'pc'] = radial_density_200
                    df['clouds_r'+str(radii[2])+'pc'] = radial_density_400
                    df['clouds_r'+str(radii[3])+'pc'] = radial_density_600
                    df['clouds_r'+str(radii[4])+'pc'] = radial_density_800
                    df['clouds_r'+str(radii[5])+'pc'] = radial_density_1000
                    df['clouds_r'+str(radii[6])+'pc'] = radial_density_1500
                    df.to_csv(fn)

In [31]:
#REAL DATA
res = [150,120,90,60]
radii = [100,200,400,600,800,1000,1500]
for k in range(len(res)):
    for i in range(len(sources)):
        real_cat = '/home/machado.35/projects/intro/sources/'+sources[i]+'/'+sources[i]+'_'+str(res[k])+'pc_cloud_stats.csv'
        if os.path.isfile(real_cat):
            cat = pd.read_csv(real_cat)
            row = int(np.where(t['name'] == sources[i])[0])
            dist = t['dist'][row]*1e6 #from Mpc to pc
            x = cat['corr_x']
            y = cat['y']
            mindist = np.zeros_like(x)*np.nan
            mindist3 = np.zeros_like(x)*np.nan
            mindist5 = np.zeros_like(x)*np.nan
            radial_density_100 = np.zeros_like(x)*np.nan
            radial_density_200 = np.zeros_like(x)*np.nan
            radial_density_400 = np.zeros_like(x)*np.nan
            radial_density_600 = np.zeros_like(x)*np.nan
            radial_density_800 = np.zeros_like(x)*np.nan
            radial_density_1000 = np.zeros_like(x)*np.nan
            radial_density_1500 = np.zeros_like(x)*np.nan
            
            mindist= spacing_calc.pair_dist_calc(x=x,y=y, neighbor_num=1) #1st nearest
            mindist3= spacing_calc.pair_dist_calc(x=x,y=y, neighbor_num=3) #3rd nearest
            mindist5= spacing_calc.pair_dist_calc(x=x,y=y, neighbor_num=5) #5th nearest
            radial_density_100 = spacing_calc.rad_density(x=x,y=y, rad=radii[0], galdist=dist)
            radial_density_200 = spacing_calc.rad_density(x=x,y=y, rad=radii[1], galdist=dist)
            radial_density_400 = spacing_calc.rad_density(x=x,y=y, rad=radii[2], galdist=dist)
            radial_density_600 = spacing_calc.rad_density(x=x,y=y, rad=radii[3], galdist=dist)
            radial_density_800 = spacing_calc.rad_density(x=x,y=y, rad=radii[4], galdist=dist)
            radial_density_1000 = spacing_calc.rad_density(x=x,y=y, rad=radii[5], galdist=dist)
            radial_density_1500 = spacing_calc.rad_density(x=x,y=y, rad=radii[6], galdist=dist)
            
            cat['nn_dist'] = np.deg2rad(mindist) * dist
            cat['nn3_dist'] = np.deg2rad(mindist3) * dist
            cat['nn5_dist'] = np.deg2rad(mindist5) * dist
            cat['clouds_r'+str(radii[0])+'pc'] = radial_density_100
            cat['clouds_r'+str(radii[1])+'pc'] = radial_density_200
            cat['clouds_r'+str(radii[2])+'pc'] = radial_density_400
            cat['clouds_r'+str(radii[3])+'pc'] = radial_density_600
            cat['clouds_r'+str(radii[4])+'pc'] = radial_density_800
            cat['clouds_r'+str(radii[5])+'pc'] = radial_density_1000
            cat['clouds_r'+str(radii[6])+'pc'] = radial_density_1500
            cat.to_csv(real_cat)